## Joins

In [1]:
#Importing the library
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("day4").getOrCreate()


23/09/04 00:39:09 WARN Utils: Your hostname, anujs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.67 instead (on interface en0)
23/09/04 00:39:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/04 00:39:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/04 00:39:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


23/09/04 00:39:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors



Question: You are given two DataFrames: employees_df and departments_df, which contain information about employees and their respective departments. The schema for the DataFrames is as follows:

employees_df schema:
|-- employee_id: integer (nullable = true)
|-- employee_name: string (nullable = true)
|-- department_id: integer (nullable = true)

departments_df schema:

|-- department_id: integer (nullable = true)
|-- department_name: string (nullable = true)

Employees DataFrame:
                                                                                
+-----------+-------------+-------------+
|employee_id|employee_name|department_id|
+-----------+-------------+-------------+
|1          |Pallavi mam  |101          |
|2          |Bob          |102          |
|3          |Cathy        |101          |
|4          |David        |103          |
|5          |Amrit Sir    |104          |
|6          |Alice        |null         |
|7          |Eva          |null         |
|8          |Frank        |110          |
|9          |Grace        |109          |
|10         |Henry        |null         |
+-----------+-------------+-------------+



Departments DataFrame:
+-------------+------------------------+
|department_id|department_name         |
+-------------+------------------------+
|101          |HR                      |
|102          |Engineering             |
|103          |Finance                 |
|104          |Marketing               |
|105          |Operations              |
|106          |null                    |
|107          |Operations              |
|108          |Production              |
|null         |Finance                 |
|110          |Research and Development|
+-------------+----------------------


In [ ]:
#Expected_solution

Employees DataFrame:
+-----------+-------------+-------------+
|employee_id|employee_name|department_id|
+-----------+-------------+-------------+
|1          |Pallavi mam  |101          |
|2          |Bob          |102          |
|3          |Cathy        |101          |
|4          |David        |103          |
|5          |Amrit Sir    |104          |
|6          |Alice        |null         |
|7          |Eva          |null         |
|8          |Frank        |110          |
|9          |Grace        |109          |
|10         |Henry        |null         |
+-----------+-------------+-------------+

root
 |-- employee_id: integer (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- department_id: integer (nullable = true)

Departments DataFrame:
+-------------+------------------------+
|department_id|department_name         |
+-------------+------------------------+
|101          |HR                      |
|102          |Engineering             |
|103          |Financ

In [3]:
#My_solution

#Importing all the neccessary library for SQL Types
from pyspark.sql.types import StructField,StructType,StringType,IntegerType

In [11]:
#Importing the data to create Schema & DataFrame

employee_data = [(1,'Pallavi mam',101),
                 (2,'Bob',102),
                 (3,'Cathy',101),
                 (4,'David',103),
                 (5,'Amrit Sir',104),
                 (6,'Alice',None),
                 (7,'Eva',None),
                 (8,'Frank',110),
                 (9,'Grace',109),
                 (10,'Henry',None)]

Department_Data = [(101,'Hr'),
                   (102,'Engineering'),
                   (103,'Finance'),
                   (104,'Marketing'),
                   (105,'Operation'),
                   (106,None),
                   (107,'Operations'),
                   (108,'Production'),
                   (None,'Finance'),
                   (110,'Research and Development')]

employee_schema = StructType([
    StructField("Employee_Id",IntegerType(),True),
    StructField("Employee_name",StringType(),True),
    StructField("department_id",IntegerType(),True)
])

department_schema = StructType([
    StructField("department_id",IntegerType(),True),
    StructField("department_name",StringType(),True)
])

employee_df  = spark.createDataFrame(data=employee_data,schema=employee_schema)
department_df = spark.createDataFrame(data=Department_Data, schema=department_schema)

print("Employee Dataframe")
employee_df.printSchema()
employee_df.show()

print("Department Dataframe")
department_df.printSchema()
department_df.show(truncate=False)

Employee Dataframe
root
 |-- Employee_Id: integer (nullable = true)
 |-- Employee_name: string (nullable = true)
 |-- department_id: integer (nullable = true)

+-----------+-------------+-------------+
|Employee_Id|Employee_name|department_id|
+-----------+-------------+-------------+
|          1|  Pallavi mam|          101|
|          2|          Bob|          102|
|          3|        Cathy|          101|
|          4|        David|          103|
|          5|    Amrit Sir|          104|
|          6|        Alice|         null|
|          7|          Eva|         null|
|          8|        Frank|          110|
|          9|        Grace|          109|
|         10|        Henry|         null|
+-----------+-------------+-------------+

Department Dataframe
root
 |-- department_id: integer (nullable = true)
 |-- department_name: string (nullable = true)

+-------------+------------------------+
|department_id|department_name         |
+-------------+------------------------+
|101    

### Join Expressions

Question: How can you combine the employees_df and departments_df DataFrames based on the common "department_id" column to get a combined DataFrame with employee names and their respective department names?

In [ ]:
#Expected_solution

+-------------+-----------+-------------+--------------------+
|department_id|employee_id|employee_name|     department_name|
+-------------+-----------+-------------+--------------------+
|          101|          1|  Pallavi mam|                  HR|
|          101|          3|        Cathy|                  HR|
|          102|          2|          Bob|         Engineering|
|          103|          4|        David|             Finance|
|          104|          5|    Amrit Sir|           Marketing|
|          110|          8|        Frank|Research and Deve...|
+-------------+-----------+-------------+--------------------+



In [12]:
#My_solution

#Combining the employees_df and departments_df DataFrames based on the common "department_id" column
# to get a combined DataFrame with employee names and their respective department names

joinExpression = department_df['department_id'] == employee_df['department_id']

department_df.join(employee_df, joinExpression).select(department_df['department_id'],employee_df['employee_id'],employee_df['employee_name'],department_df['department_name']).show()

+-------------+-----------+-------------+--------------------+
|department_id|employee_id|employee_name|     department_name|
+-------------+-----------+-------------+--------------------+
|          101|          1|  Pallavi mam|                  Hr|
|          101|          3|        Cathy|                  Hr|
|          102|          2|          Bob|         Engineering|
|          103|          4|        David|             Finance|
|          104|          5|    Amrit Sir|           Marketing|
|          110|          8|        Frank|Research and Deve...|
+-------------+-----------+-------------+--------------------+



### Inner Joins

Question: How can you retrieve employee names and their respective department names for employees belonging to the "Engineering" department?

In [ ]:
#Expected_solution

+-------------+---------------+
|employee_name|department_name|
+-------------+---------------+
|          Bob|    Engineering|
+-------------+---------------+



In [13]:
#My_solution

#retrieving employee names and their respective department names 
# for employees belonging to the "Engineering" department

innerJoinExpression = department_df['department_id'] == employee_df['department_id']
engineeringExpression = department_df['department_name'] == 'Engineering'

department_df.join(employee_df, innerJoinExpression).select(employee_df['employee_name'], department_df['department_name']).where(engineeringExpression).show()


+-------------+---------------+
|employee_name|department_name|
+-------------+---------------+
|          Bob|    Engineering|
+-------------+---------------+



### Outer Joins

Question: Retrieve a DataFrame that contains all employees along with their department names. If an employee doesn't have a department assigned, display "No Department".

In [ ]:
#Expected_solution

+-------------+--------------------+
|employee_name|     department_name|
+-------------+--------------------+
|        Alice|       No Department|
|          Eva|       No Department|
|        Henry|       No Department|
|No Department|             Finance|
|  Pallavi mam|                  HR|
|        Cathy|                  HR|
|          Bob|         Engineering|
|        David|             Finance|
|    Amrit Sir|           Marketing|
|No Department|          Operations|
|No Department|       No Department|
|No Department|          Operations|
|No Department|          Production|
|        Grace|       No Department|
|        Frank|Research and Deve...|
+-------------+--------------------+



In [14]:
#My_solution

joinType = 'outer'

#Outer joined two dataframes which returned every value possible and null if any value is not present
outer_joined_df = department_df.join(employee_df, joinExpression, joinType).select(employee_df['employee_name'],department_df['department_name'])

#filled null with default value as No Department
No_dept_filled = outer_joined_df.na.fill('No Department')
No_dept_filled.show(truncate=False)

+-------------+------------------------+
|employee_name|department_name         |
+-------------+------------------------+
|No Department|Finance                 |
|Alice        |No Department           |
|Eva          |No Department           |
|Henry        |No Department           |
|Pallavi mam  |Hr                      |
|Cathy        |Hr                      |
|Bob          |Engineering             |
|David        |Finance                 |
|Amrit Sir    |Marketing               |
|No Department|Operation               |
|No Department|No Department           |
|No Department|Operations              |
|No Department|Production              |
|Grace        |No Department           |
|Frank        |Research and Development|
+-------------+------------------------+



### Left Outer Joins

Question: List all employees along with their department names. If an employee doesn't have a department assigned, display "No Department".

In [ ]:
#Expected_solution

+-------------+--------------------+
|employee_name|     department_name|
+-------------+--------------------+
|  Pallavi mam|                  HR|
|          Bob|         Engineering|
|        Cathy|                  HR|
|        David|             Finance|
|    Amrit Sir|           Marketing|
|        Alice|       No Department|
|          Eva|       No Department|
|        Frank|Research and Deve...|
|        Grace|       No Department|
|        Henry|       No Department|
+-------------+--------------------+



In [19]:
#My_solution

joinType = 'left_outer'
employee_df.join(department_df,joinExpression,joinType).select(employee_df['employee_name'],department_df['department_name']).na.fill('no Department').show()

+-------------+--------------------+
|employee_name|     department_name|
+-------------+--------------------+
|  Pallavi mam|                  Hr|
|          Bob|         Engineering|
|        Cathy|                  Hr|
|        David|             Finance|
|    Amrit Sir|           Marketing|
|        Alice|       no Department|
|          Eva|       no Department|
|        Frank|Research and Deve...|
|        Henry|       no Department|
|        Grace|       no Department|
+-------------+--------------------+



### Right Outer Joins

Question: Display a list of departments along with employee names. If a department has no employees, display "No Employees".



In [26]:
#My_solution

"""
    I guess there is problem in the given expected output.
    The expected output also gives the null value in the department column as "No Employee", but i guess No employee should only be given
    in employee column so did so.

    for the result to be as given in assignment we can remove subset=[] from fill()
"""

joinType = 'right_outer'

employee_df.join(department_df,joinExpression,joinType).select(department_df['department_name'],employee_df['employee_name']).na.fill('No Employee', subset=['employee_name']).show()

+--------------------+-------------+
|     department_name|employee_name|
+--------------------+-------------+
|                  Hr|        Cathy|
|                  Hr|  Pallavi mam|
|         Engineering|          Bob|
|             Finance|        David|
|           Operation|  No Employee|
|           Marketing|    Amrit Sir|
|                null|  No Employee|
|          Operations|  No Employee|
|          Production|  No Employee|
|             Finance|  No Employee|
|Research and Deve...|        Frank|
+--------------------+-------------+



### Left Semi Joins

Question: Retrieve a DataFrame that includes employee names for departments that have employees.



In [ ]:
#Expected_solution

+-------------+
|employee_name|
+-------------+
|  Pallavi mam|
|        Cathy|
|          Bob|
|        David|
|    Amrit Sir|
|        Frank|
+-------------+



In [27]:
#My_solution

joinType = 'left_semi'

employee_df.join(department_df,joinExpression,joinType).select(employee_df['employee_name']).show()

+-------------+
|employee_name|
+-------------+
|  Pallavi mam|
|        Cathy|
|          Bob|
|        David|
|    Amrit Sir|
|        Frank|
+-------------+



### Left Anti Joins

Question: Find the employees who don't belong to any department.

In [ ]:
#Expected_solution

+-------------+
|employee_name|
+-------------+
|        Alice|
|          Eva|
|        Grace|
|        Henry|
+-------------+



In [28]:
#My_solution

joinType = 'left_anti'

employee_df.join(department_df,joinExpression,joinType).select(employee_df['employee_name']).show()


+-------------+
|employee_name|
+-------------+
|        Alice|
|          Eva|
|        Henry|
|        Grace|
+-------------+



### Cross (Cartesian) Joins

Question: Create a DataFrame that contains all possible combinations of employees and departments.

In [ ]:
#Expected_solution

+-----------+-------------+-------------+-------------+--------------------+
|employee_id|employee_name|department_id|department_id|     department_name|
+-----------+-------------+-------------+-------------+--------------------+
|          1|  Pallavi mam|          101|          101|                  HR|
|          1|  Pallavi mam|          101|          102|         Engineering|
|          1|  Pallavi mam|          101|          103|             Finance|
|          1|  Pallavi mam|          101|          104|           Marketing|
|          1|  Pallavi mam|          101|          105|          Operations|
|          1|  Pallavi mam|          101|          106|                null|
|          1|  Pallavi mam|          101|          107|          Operations|
|          1|  Pallavi mam|          101|          108|          Production|
|          1|  Pallavi mam|          101|         null|             Finance|
|          1|  Pallavi mam|          101|          110|Research and Deve...|

In [29]:
#My_solution

joinType = 'cross'
employee_df.crossJoin(department_df).show()

+-----------+-------------+-------------+-------------+--------------------+
|Employee_Id|Employee_name|department_id|department_id|     department_name|
+-----------+-------------+-------------+-------------+--------------------+
|          1|  Pallavi mam|          101|          101|                  Hr|
|          1|  Pallavi mam|          101|          102|         Engineering|
|          1|  Pallavi mam|          101|          103|             Finance|
|          1|  Pallavi mam|          101|          104|           Marketing|
|          1|  Pallavi mam|          101|          105|           Operation|
|          1|  Pallavi mam|          101|          106|                null|
|          1|  Pallavi mam|          101|          107|          Operations|
|          1|  Pallavi mam|          101|          108|          Production|
|          1|  Pallavi mam|          101|         null|             Finance|
|          1|  Pallavi mam|          101|          110|Research and Deve...|

In [ ]:
#Expected_solution